In [2]:
import nltk
import random

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Fonction pour supprimer des unigrammes
def drop_unigrams_arabic(text, drop_prob=0.2):
    words = nltk.word_tokenize(text)
    modified_text = []
    for word in words:
        if random.uniform(0, 1) > drop_prob:
            modified_text.append(word)
    return ' '.join(modified_text)

In [5]:
# Fonction pour ajouter des unigrammes
def add_unigrams_arabic(text, add_prob=0.2, unigrams=['و', 'في', 'على']):
    words = nltk.word_tokenize(text)
    modified_text = []
    for word in words:
        modified_text.append(word)
        if random.uniform(0, 1) < add_prob:
            random_unigram = random.choice(unigrams)
            modified_text.append(random_unigram)
    return ' '.join(modified_text)

In [6]:
# Votre texte original en arabe
original_text_arabic = "القط نائم على السجادة."

# Augmentation du texte en supprimant des unigrammes
augmented_text_1_arabic = drop_unigrams_arabic(original_text_arabic)

# Augmentation du texte en ajoutant des unigrammes
augmented_text_2_arabic = add_unigrams_arabic(original_text_arabic)

In [7]:
print(augmented_text_1_arabic)
print(augmented_text_2_arabic)

القط نائم على السجادة .
القط نائم على على السجادة .


In [8]:
#create lidst of runction 
noise = [drop_unigrams_arabic, add_unigrams_arabic]


In [9]:
# Randomly select a function and execute it
selected_noise = random.choice(noise)

In [10]:
import pandas as pd

df = pd.read_csv('data.csv')
df.head()

,comment,type
0,يعطيكم الصحة,normal
1,عشي عندهما رجال,normal
2,نبيك مغتصب الاطفال المنيوك,aggressive
3,بشير فار حبس خارج ثالثة ابتيدائي جاي اتناقش ما...,normal
4,مول القناة دزيزي جنيت تفرجت الفديو تاعك ولدي ت...,normal


In [11]:
df['preprocessed_txt'] = df['comment'].apply(selected_noise)

In [12]:
df.head()

,comment,type,preprocessed_txt
0,يعطيكم الصحة,normal,يعطيكم الصحة
1,عشي عندهما رجال,normal,عشي عندهما
2,نبيك مغتصب الاطفال المنيوك,aggressive,مغتصب المنيوك
3,بشير فار حبس خارج ثالثة ابتيدائي جاي اتناقش ما...,normal,بشير فار حبس ثالثة ابتيدائي جاي مايا عيب اعليك...
4,مول القناة دزيزي جنيت تفرجت الفديو تاعك ولدي ت...,normal,مول القناة دزيزي تفرجت الفديو تاعك ولدي تعرفو ...


In [13]:
df['aggressive'] = df['type'].apply(lambda x: 1 if x =='aggressive' else  0)

In [14]:
#train the model
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(
    df.preprocessed_txt,
    df.aggressive,
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
   stratify=df.type
)

In [15]:
# create bag of words using Countvectorizer
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train)
X_train_cv

<3524x13689 sparse matrix of type '<class 'numpy.int64'>'
	with 32231 stored elements in Compressed Sparse Row format>

In [16]:
#train the naive bayes model
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [17]:
X_test_cv = v.transform(X_test)

In [18]:
    #Evaluate Performance
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80       441
           1       0.79      0.82      0.80       441

    accuracy                           0.80       882
   macro avg       0.80      0.80      0.80       882
weighted avg       0.80      0.80      0.80       882



In [20]:
#add type in liste
type_l = []
for i in df["type"]:
    type_l.append(i)
#create data to mix  data and type
data_l = {'comment': df["preprocessed_txt"], 'type': type_l}

#creta data frame
dt = pd.DataFrame(data_l)

# Save the data frame to a CSV file
dt.to_csv("aug_noise.csv", index=False)

In [22]:
# Load data from the first CSV file
df1 = pd.read_csv('data.csv')

# Load data from the second CSV file
df2 = pd.read_csv('aug_noise.csv')

# Concatenate the two DataFrames vertically
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('aug_all_noise.csv', index=False)

In [23]:
df3 = pd.read_csv("aug_all_noise.csv")
df3.head()
print(len(df3))

8812


In [24]:
df3['aggressive'] = df3['type'].apply(lambda x: 1 if x =='aggressive' else  0)
df3.head()

,comment,type,aggressive
0,يعطيكم الصحة,normal,0
1,عشي عندهما رجال,normal,0
2,نبيك مغتصب الاطفال المنيوك,aggressive,1
3,بشير فار حبس خارج ثالثة ابتيدائي جاي اتناقش ما...,normal,0
4,مول القناة دزيزي جنيت تفرجت الفديو تاعك ولدي ت...,normal,0


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df3.comment,df3.aggressive,test_size=0.2)

In [29]:
h = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values.astype('U'))
X_train_cv

<7049x16884 sparse matrix of type '<class 'numpy.int64'>'
	with 73135 stored elements in Compressed Sparse Row format>

In [30]:
#train the naive bayes model
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [32]:
X_test_cv = v.transform(X_test.values.astype('U'))


#Evaluate Performance
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91       898
           1       0.89      0.93      0.91       865

    accuracy                           0.91      1763
   macro avg       0.91      0.91      0.91      1763
weighted avg       0.91      0.91      0.91      1763

